In [ ]:
#  !pip install evaluate

In [ ]:
# !pip install bert_score

In [1]:
from evaluate import load
import csv
import re

def clean_text(text):
    text = text.replace('\n', ' ')  # 換行轉空格
    text = re.sub(r'\*+', '', text) # 移除星號（markdown）
    return text.strip()

# input_file = './2_GPT_Ranking/output_ranking_100_4o.csv'
input_file = '/home/yuu0223/KG-Prompting-Based-on-Community-Search/output/MindMap/Final_Q714_Gemini1.5/20250505_Gemini15.csv'
output_file = './BERTscore/0507_MM_test.csv'

bertscore = load("bertscore")

with open(input_file,'r',newline="") as f_input, open(output_file, 'a+', newline='') as f_output:
  reader = csv.reader(f_input)
  writer = csv.writer(f_output)

  header = next(reader)
  output_num = 3 ###
  output_columns = []
  for i in range(1, output_num+1):
    output_columns.append(f"output{i}_precision")
    output_columns.append(f"output{i}_recall")
    output_columns.append(f"output{i}_f1")
  header.extend(output_columns)
  writer.writerow(header)

  for row in reader:

    output1_text = [clean_text(row[3])]
    output2_text = [clean_text(row[4])]
    output3_text = [clean_text(row[5])] ###

    references = [clean_text(row[2])]

    results1 = bertscore.compute(predictions=output1_text, references=references, model_type="distilbert-base-uncased")
    results2 = bertscore.compute(predictions=output2_text, references=references, model_type="distilbert-base-uncased")
    results3 = bertscore.compute(predictions=output3_text, references=references, model_type="distilbert-base-uncased") ###

    print(results1["precision"][0],results1["recall"][0],results1["f1"][0])

    row.extend([results1["precision"][0],results1["recall"][0],results1["f1"][0],
                results2["precision"][0],results2["recall"][0],results2["f1"][0],
                results3["precision"][0],results3["recall"][0],results3["f1"][0]]) ###
    writer.writerow(row)

/home/yuu0223/KG-Prompting-Based-on-Community-Search/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/yuu0223/KG-Prompting-Based-on-Community-Search/venv/lib/python3.10/site-packages/torch/cuda/__init__.py:546: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


0.71882563829422 0.8500621318817139 0.7789549827575684
0.7460775375366211 0.8365325927734375 0.7887200117111206
0.8058720827102661 0.8524618148803711 0.8285124897956848
0.7484262585639954 0.8195791840553284 0.7823883295059204
0.7707462310791016 0.821470320224762 0.795300304889679
0.7217960357666016 0.8415911197662354 0.7771039605140686
0.7689953446388245 0.7722246050834656 0.7706066370010376
0.75119549036026 0.8278825283050537 0.7876769304275513
0.7208319306373596 0.8198527097702026 0.7671602368354797
0.7536356449127197 0.8321268558502197 0.790938675403595
0.7576373815536499 0.7954161763191223 0.7760672569274902
0.785919725894928 0.7966475486755371 0.7912473082542419
0.7879607677459717 0.837730348110199 0.8120837211608887
0.7345361709594727 0.8524982333183289 0.7891331911087036
0.7598362565040588 0.7998598217964172 0.779334545135498
0.7270556092262268 0.8316605091094971 0.7758480906486511
0.736464262008667 0.8410439491271973 0.785287618637085
0.7290985584259033 0.8507438898086548 0.785

In [3]:
import csv

# 定義檔案名稱
input_file = './BERTscore/0507_MM_test.csv'

# 初始化欄位名稱與累加器
# columns = [
#     "output1_precision", "output1_recall", "output1_f1",
#     "output2_precision", "output2_recall", "output2_f1",
#     "output3_precision", "output3_recall", "output3_f1"
# ]

columns = []
output_num = 3 ###
for i in range(1, output_num+1):
    columns.append(f"output{i}_precision")
    columns.append(f"output{i}_recall")
    columns.append(f"output{i}_f1")

sums = {col: 0.0 for col in columns}  # 每個欄位的累加和
counts = {col: 0 for col in columns}  # 每個欄位的計數器

# 讀取檔案並計算總和與計數
with open(input_file, 'r', newline="") as f_input:
    reader = csv.DictReader(f_input)  # 使用 DictReader 方便按欄位名稱讀取
    for row in reader:
        for col in columns:
            if row[col]:  # 確保欄位非空
                sums[col] += float(row[col])  # 累加欄位值
                counts[col] += 1  # 增加計數

# 計算平均值
averages = {col: (sums[col] / counts[col] if counts[col] > 0 else 0) for col in columns}
print("counts:", counts)

# 印出平均值
print("每個欄位的平均值：")
for col, avg in averages.items():
    print(f"{col}: {round(avg, 3)}")

counts: {'output1_precision': 714, 'output1_recall': 714, 'output1_f1': 714, 'output2_precision': 714, 'output2_recall': 714, 'output2_f1': 714, 'output3_precision': 714, 'output3_recall': 714, 'output3_f1': 714}
每個欄位的平均值：
output1_precision: 0.745
output1_recall: 0.815
output1_f1: 0.778
output2_precision: 0.741
output2_recall: 0.809
output2_f1: 0.773
output3_precision: 0.743
output3_recall: 0.813
output3_f1: 0.776


## Large Model

In [3]:
from evaluate import load
import csv
import re

def clean_text(text):
    text = text.replace('\n', ' ')  # 換行轉空格
    text = re.sub(r'\*+', '', text) # 移除星號（markdown）
    return text.strip()


# input_file = './2_GPT_Ranking/output_ranking_100_4o.csv'
input_file = './1_merge_file/merged_output_Q714_Gemini15Flash_0506_new.csv'
output_file = './BERTscore/0506_Q714_Gemini15_new.csv'

bertscore = load("bertscore")

with open(input_file,'r',newline="") as f_input, open(output_file, 'a+', newline='') as f_output:
  reader = csv.reader(f_input)
  writer = csv.writer(f_output)

  header = next(reader)
  output_num = 8 ###
  output_columns = []
  for i in range(1, output_num+1):
    output_columns.append(f"output{i}_precision")
    output_columns.append(f"output{i}_recall")
    output_columns.append(f"output{i}_f1")
  header.extend(output_columns)
  writer.writerow(header)

  for row in reader:

    output1_text = [clean_text(row[3].strip("\n"))]
    output2_text = [clean_text(row[4].strip("\n"))]
    output3_text = [clean_text(row[5].strip("\n"))]
    output4_text = [clean_text(row[6].strip("\n"))]
    output5_text = [clean_text(row[7].strip("\n"))]
    output6_text = [clean_text(row[8].strip("\n"))] 
    output7_text = [clean_text(row[9].strip("\n"))]
    output8_text = [clean_text(row[10].strip("\n"))] ###


    references = [clean_text(row[2].strip("\n"))]

    results1 = bertscore.compute(predictions=output1_text, references=references, model_type="distilbert-base-uncased")
    results2 = bertscore.compute(predictions=output2_text, references=references, model_type="distilbert-base-uncased")
    results3 = bertscore.compute(predictions=output3_text, references=references, model_type="distilbert-base-uncased")
    results4 = bertscore.compute(predictions=output4_text, references=references, model_type="distilbert-base-uncased")
    results5 = bertscore.compute(predictions=output5_text, references=references, model_type="distilbert-base-uncased")
    results6 = bertscore.compute(predictions=output6_text, references=references, model_type="distilbert-base-uncased")
    results7 = bertscore.compute(predictions=output7_text, references=references, model_type="distilbert-base-uncased")
    results8 = bertscore.compute(predictions=output8_text, references=references, model_type="distilbert-base-uncased") ###

    print(results1["precision"][0],results1["recall"][0],results1["f1"][0])

    row.extend([results1["precision"][0],results1["recall"][0],results1["f1"][0],
                results2["precision"][0],results2["recall"][0],results2["f1"][0],
                results3["precision"][0],results3["recall"][0],results3["f1"][0],
                results4["precision"][0],results4["recall"][0],results4["f1"][0],
                results5["precision"][0],results5["recall"][0],results5["f1"][0],
                results6["precision"][0],results6["recall"][0],results6["f1"][0],
                results7["precision"][0],results7["recall"][0],results7["f1"][0],
                results8["precision"][0],results8["recall"][0],results8["f1"][0]
                ]) ###
    writer.writerow(row)


0.754697859287262 0.8163438439369202 0.7843114137649536
0.7766966223716736 0.8266309499740601 0.8008862137794495
0.7526853084564209 0.8527168035507202 0.7995846271514893
0.7454200983047485 0.8274726271629333 0.7843061685562134
0.7564804553985596 0.8573637008666992 0.8037689328193665
0.7392120957374573 0.8364090323448181 0.7848126292228699
0.7194555997848511 0.862264096736908 0.7844130396842957
0.7795645594596863 0.8563290238380432 0.816145658493042
0.7619061470031738 0.8779003620147705 0.8158007264137268
0.7500924468040466 0.831519365310669 0.788709819316864
0.728500485420227 0.8071422576904297 0.7658076882362366
0.7411253452301025 0.8139733076095581 0.7758430242538452
0.7719757556915283 0.850930392742157 0.8095324039459229
0.7519016265869141 0.7833255529403687 0.7672920227050781
0.7788350582122803 0.7972383499145508 0.7879292964935303
0.7440304756164551 0.8357546329498291 0.7872297763824463
0.7422954440116882 0.8276593089103699 0.7826566100120544
0.7175445556640625 0.872784435749054 0

In [4]:
import csv

# 定義檔案名稱
input_file = './BERTscore/0506_Q714_Gemini15_new.csv'

# 初始化欄位名稱與累加器
# columns = [
#     "output1_precision", "output1_recall", "output1_f1",
#     "output2_precision", "output2_recall", "output2_f1",
#     "output3_precision", "output3_recall", "output3_f1"
# ]

columns = []
output_num = 8 ###
for i in range(1, output_num+1):
    columns.append(f"output{i}_precision")
    columns.append(f"output{i}_recall")
    columns.append(f"output{i}_f1")

sums = {col: 0.0 for col in columns}  # 每個欄位的累加和
counts = {col: 0 for col in columns}  # 每個欄位的計數器

# 讀取檔案並計算總和與計數
with open(input_file, 'r', newline="") as f_input:
    reader = csv.DictReader(f_input)  # 使用 DictReader 方便按欄位名稱讀取
    for row in reader:
        for col in columns:
            if row[col]:  # 確保欄位非空
                sums[col] += float(row[col])  # 累加欄位值
                counts[col] += 1  # 增加計數

# 計算平均值
averages = {col: (sums[col] / counts[col] if counts[col] > 0 else 0) for col in columns}

# 印出平均值
print("每個欄位的平均值：")
for col, avg in averages.items():
    print(f"{col}: {round(avg, 3)}")

每個欄位的平均值：
output1_precision: 0.749
output1_recall: 0.822
output1_f1: 0.784
output2_precision: 0.75
output2_recall: 0.823
output2_f1: 0.784
output3_precision: 0.749
output3_recall: 0.82
output3_f1: 0.782
output4_precision: 0.749
output4_recall: 0.821
output4_f1: 0.783
output5_precision: 0.749
output5_recall: 0.822
output5_f1: 0.784
output6_precision: 0.749
output6_recall: 0.822
output6_f1: 0.783
output7_precision: 0.745
output7_recall: 0.815
output7_f1: 0.778
output8_precision: 0.728
output8_recall: 0.821
output8_f1: 0.771


## Small Model

In [2]:
from evaluate import load
import csv
import re

def clean_text(text):
    text = text.replace('\n', ' ')  # 換行轉空格
    text = re.sub(r'\*+', '', text) # 移除星號（markdown）
    return text.strip()


# input_file = './2_GPT_Ranking/output_ranking_100_4o.csv'
input_file = './1_merge_file/merged_output_Q714_Gemma3-1B_0520.csv'
output_file = './BERTscore/0520_Q714_Gemma3-1B.csv'

bertscore = load("bertscore")

with open(input_file,'r',newline="") as f_input, open(output_file, 'a+', newline='') as f_output:
  reader = csv.reader(f_input)
  writer = csv.writer(f_output)

  header = next(reader)
  output_num = 8 ###
  output_columns = []
  for i in range(1, output_num+1):
    output_columns.append(f"output{i}_precision")
    output_columns.append(f"output{i}_recall")
    output_columns.append(f"output{i}_f1")
  header.extend(output_columns)
  writer.writerow(header)

  for row in reader:

    output1_text = [clean_text(row[3].strip("\n"))]
    output2_text = [clean_text(row[4].strip("\n"))]
    output3_text = [clean_text(row[5].strip("\n"))]
    output4_text = [clean_text(row[6].strip("\n"))]
    output5_text = [clean_text(row[7].strip("\n"))]
    output6_text = [clean_text(row[8].strip("\n"))] 
    output7_text = [clean_text(row[9].strip("\n"))]
    output8_text = [clean_text(row[10].strip("\n"))] ###


    references = [clean_text(row[2].strip("\n"))]

    results1 = bertscore.compute(predictions=output1_text, references=references, model_type="distilbert-base-uncased")
    results2 = bertscore.compute(predictions=output2_text, references=references, model_type="distilbert-base-uncased")
    results3 = bertscore.compute(predictions=output3_text, references=references, model_type="distilbert-base-uncased")
    results4 = bertscore.compute(predictions=output4_text, references=references, model_type="distilbert-base-uncased")
    results5 = bertscore.compute(predictions=output5_text, references=references, model_type="distilbert-base-uncased")
    results6 = bertscore.compute(predictions=output6_text, references=references, model_type="distilbert-base-uncased")
    results7 = bertscore.compute(predictions=output7_text, references=references, model_type="distilbert-base-uncased")
    results8 = bertscore.compute(predictions=output8_text, references=references, model_type="distilbert-base-uncased") ###

    print(results1["precision"][0],results1["recall"][0],results1["f1"][0])

    row.extend([results1["precision"][0],results1["recall"][0],results1["f1"][0],
                results2["precision"][0],results2["recall"][0],results2["f1"][0],
                results3["precision"][0],results3["recall"][0],results3["f1"][0],
                results4["precision"][0],results4["recall"][0],results4["f1"][0],
                results5["precision"][0],results5["recall"][0],results5["f1"][0],
                results6["precision"][0],results6["recall"][0],results6["f1"][0],
                results7["precision"][0],results7["recall"][0],results7["f1"][0],
                results8["precision"][0],results8["recall"][0],results8["f1"][0]
                ]) ###
    writer.writerow(row)


0.7770794630050659 0.8191424012184143 0.7975566983222961
0.7671828269958496 0.8106353282928467 0.7883107662200928
0.7643805742263794 0.8559694290161133 0.8075864911079407
0.7492769360542297 0.8053580522537231 0.7763059735298157
0.725214421749115 0.8621244430541992 0.7877650260925293
0.7775121927261353 0.8472128510475159 0.810867428779602
0.7292569279670715 0.8414026498794556 0.7813261151313782
0.7750738859176636 0.8401444554328918 0.8062983751296997
0.723784863948822 0.8254269957542419 0.7712715864181519
0.7652770280838013 0.8244552612304688 0.7937647104263306
0.7324737310409546 0.8246008157730103 0.7758118510246277
0.7551701068878174 0.8331617116928101 0.7922511100769043
0.7415174245834351 0.8179501891136169 0.7778607606887817
0.7756266593933105 0.8196703195571899 0.7970405220985413
0.7642112970352173 0.7988160848617554 0.7811306118965149
0.7645751237869263 0.8326270580291748 0.7971513271331787
0.7477881908416748 0.8256773352622986 0.7848049402236938
0.7469245791435242 0.8535434007644

In [1]:
import csv

# 定義檔案名稱
input_file = './BERTscore/0520_Q714_Gemma3-1B.csv'

# 初始化欄位名稱與累加器
# columns = [
#     "output1_precision", "output1_recall", "output1_f1",
#     "output2_precision", "output2_recall", "output2_f1",
#     "output3_precision", "output3_recall", "output3_f1"
# ]

columns = []
output_num = 8 ###
for i in range(1, output_num+1):
    columns.append(f"output{i}_precision")
    columns.append(f"output{i}_recall")
    columns.append(f"output{i}_f1")

sums = {col: 0.0 for col in columns}  # 每個欄位的累加和
counts = {col: 0 for col in columns}  # 每個欄位的計數器

# 讀取檔案並計算總和與計數
with open(input_file, 'r', newline="") as f_input:
    reader = csv.DictReader(f_input)  # 使用 DictReader 方便按欄位名稱讀取
    for row in reader:
        for col in columns:
            if row[col]:  # 確保欄位非空
                sums[col] += float(row[col])  # 累加欄位值
                counts[col] += 1  # 增加計數

# 計算平均值
averages = {col: (sums[col] / counts[col] if counts[col] > 0 else 0) for col in columns}

# 印出平均值
print("每個欄位的平均值：")
for col, avg in averages.items():
    print(f"{col}: {round(avg, 5)}")

每個欄位的平均值：
output1_precision: 0.75219
output1_recall: 0.80865
output1_f1: 0.77903
output2_precision: 0.75279
output2_recall: 0.8098
output2_f1: 0.77989
output3_precision: 0.75257
output3_recall: 0.81029
output3_f1: 0.78004
output4_precision: 0.75145
output4_recall: 0.81189
output4_f1: 0.7802
output5_precision: 0.75194
output5_recall: 0.80827
output5_f1: 0.77872
output6_precision: 0.75248
output6_recall: 0.80927
output6_f1: 0.77949
output7_precision: 0.76801
output7_recall: 0.79432
output7_f1: 0.78046
output8_precision: 0.72088
output8_recall: 0.81029
output8_f1: 0.76269


In [6]:
from evaluate import load
import csv
import re

def clean_text(text):
    text = text.replace('\n', ' ')  # 換行轉空格
    text = re.sub(r'\*+', '', text) # 移除星號（markdown）
    return text.strip()


# input_file = './2_GPT_Ranking/output_ranking_100_4o.csv'
input_file = '../output/MindMap/Final_Q714_Gemma3-4B/20250513.csv'
output_file = './BERTscore/0514_MM_Gemma3-4B.csv'

bertscore = load("bertscore")

with open(input_file,'r',newline="") as f_input, open(output_file, 'a+', newline='') as f_output:
  reader = csv.reader(f_input)
  writer = csv.writer(f_output)

  header = next(reader)
  output_num = 1 ###
  output_columns = []
  for i in range(1, output_num+1):
    output_columns.append(f"output{i}_precision")
    output_columns.append(f"output{i}_recall")
    output_columns.append(f"output{i}_f1")
  header.extend(output_columns)
  writer.writerow(header)

  for row in reader:

    output1_text = [clean_text(row[3].strip("\n"))]
    


    references = [clean_text(row[2].strip("\n"))]

    results1 = bertscore.compute(predictions=output1_text, references=references, model_type="distilbert-base-uncased")
    

    print(results1["precision"][0],results1["recall"][0],results1["f1"][0])

    row.extend([results1["precision"][0],results1["recall"][0],results1["f1"][0]]) ###
    writer.writerow(row)


0.7289515733718872 0.8282879590988159 0.7754514217376709
0.7528814077377319 0.8336055874824524 0.7911897301673889
0.8038393259048462 0.8363603949546814 0.8197774887084961
0.7446427941322327 0.8183039426803589 0.7797375321388245
0.7783820629119873 0.8369559645652771 0.8066070079803467
0.6954997181892395 0.8514745235443115 0.7656239867210388
0.7646185159683228 0.7688047885894775 0.7667059898376465
0.770272970199585 0.8493022322654724 0.8078593611717224
0.7286855578422546 0.8241867423057556 0.7734995484352112
0.7518467307090759 0.8215634822845459 0.7851605415344238
0.774407148361206 0.8219150304794312 0.7974541783332825
0.7729092836380005 0.8124781847000122 0.792199969291687
0.7580292224884033 0.806806743144989 0.7816577553749084
0.7393231391906738 0.8499836921691895 0.7908008694648743
0.7395809888839722 0.7880173921585083 0.7630312442779541
0.697866678237915 0.8332921862602234 0.7595905065536499
0.7642687559127808 0.830319344997406 0.7959260940551758
0.7629790306091309 0.8874623775482178

In [7]:
import csv

# 定義檔案名稱
input_file = './BERTscore/0514_MM_Gemma3-4B.csv'

# 初始化欄位名稱與累加器
# columns = [
#     "output1_precision", "output1_recall", "output1_f1",
#     "output2_precision", "output2_recall", "output2_f1",
#     "output3_precision", "output3_recall", "output3_f1"
# ]

columns = []
output_num = 1 ###
for i in range(1, output_num+1):
    columns.append(f"output{i}_precision")
    columns.append(f"output{i}_recall")
    columns.append(f"output{i}_f1")

sums = {col: 0.0 for col in columns}  # 每個欄位的累加和
counts = {col: 0 for col in columns}  # 每個欄位的計數器

# 讀取檔案並計算總和與計數
with open(input_file, 'r', newline="") as f_input:
    reader = csv.DictReader(f_input)  # 使用 DictReader 方便按欄位名稱讀取
    for row in reader:
        for col in columns:
            if row[col]:  # 確保欄位非空
                sums[col] += float(row[col])  # 累加欄位值
                counts[col] += 1  # 增加計數

# 計算平均值
averages = {col: (sums[col] / counts[col] if counts[col] > 0 else 0) for col in columns}

# 印出平均值
print("每個欄位的平均值：")
for col, avg in averages.items():
    print(f"{col}: {round(avg, 3)}")

每個欄位的平均值：
output1_precision: 0.759
output1_recall: 0.828
output1_f1: 0.791


## IDF

In [ ]:
from evaluate import load
import csv

# input_file = './2_GPT_Ranking/output_ranking_100_4o.csv'
input_file = './1_merge_file/merged_output_Q714_Gemini20Flash_0417.csv'
output_file = './BERTscore/output_bert_score_Q714_Gemini20Flash_0422_IDF_new.csv'

bertscore = load("bertscore")

with open(input_file,'r',newline="") as f_input, open(output_file, 'a+', newline='') as f_output:
  reader = csv.reader(f_input)
  writer = csv.writer(f_output)

  header = next(reader)
  output_num = 8 ###
  output_columns = []
  for i in range(1, output_num+1):
    output_columns.append(f"output{i}_precision")
    output_columns.append(f"output{i}_recall")
    output_columns.append(f"output{i}_f1")
  header.extend(output_columns)
  writer.writerow(header)

  for row in reader:

    output1_text = [row[3].strip("\n")]
    output2_text = [row[4].strip("\n")]
    output3_text = [row[5].strip("\n")]
    output4_text = [row[6].strip("\n")]
    output5_text = [row[7].strip("\n")]
    output6_text = [row[8].strip("\n")] 
    output7_text = [row[9].strip("\n")]
    output8_text = [row[10].strip("\n")] ###


    references = [row[2].strip("\n")]

    results1 = bertscore.compute(predictions=output1_text, references=references, model_type="distilbert-base-uncased", idf=True)
    results2 = bertscore.compute(predictions=output2_text, references=references, model_type="distilbert-base-uncased", idf=True)
    results3 = bertscore.compute(predictions=output3_text, references=references, model_type="distilbert-base-uncased", idf=True)
    results4 = bertscore.compute(predictions=output4_text, references=references, model_type="distilbert-base-uncased", idf=True)
    results5 = bertscore.compute(predictions=output5_text, references=references, model_type="distilbert-base-uncased", idf=True)
    results6 = bertscore.compute(predictions=output6_text, references=references, model_type="distilbert-base-uncased", idf=True)
    results7 = bertscore.compute(predictions=output7_text, references=references, model_type="distilbert-base-uncased", idf=True)
    results8 = bertscore.compute(predictions=output8_text, references=references, model_type="distilbert-base-uncased", idf=True) ###

    print(results1["precision"][0],results1["recall"][0],results1["f1"][0])

    row.extend([results1["precision"][0],results1["recall"][0],results1["f1"][0],
                results2["precision"][0],results2["recall"][0],results2["f1"][0],
                results3["precision"][0],results3["recall"][0],results3["f1"][0],
                results4["precision"][0],results4["recall"][0],results4["f1"][0],
                results5["precision"][0],results5["recall"][0],results5["f1"][0],
                results6["precision"][0],results6["recall"][0],results6["f1"][0],
                results7["precision"][0],results7["recall"][0],results7["f1"][0],
                results8["precision"][0],results8["recall"][0],results8["f1"][0]
                ]) ###
    writer.writerow(row)


0.06002967432141304 nan -1.995849609375
0.04154471680521965 0.2092549055814743 0.12314002960920334
-0.044700540602207184 0.3274820148944855 0.12626011669635773
0.22568757832050323 0.22682152688503265 0.22717969119548798


KeyboardInterrupt: 

In [6]:
import csv

# 定義檔案名稱
input_file = './BERTscore/output_bert_score_Q714_Gemini20Flash_0422_IDF.csv'

# 初始化欄位名稱與累加器
# columns = [
#     "output1_precision", "output1_recall", "output1_f1",
#     "output2_precision", "output2_recall", "output2_f1",
#     "output3_precision", "output3_recall", "output3_f1"
# ]

columns = []
output_num = 8 ###
for i in range(1, output_num+1):
    columns.append(f"output{i}_precision")
    columns.append(f"output{i}_recall")
    columns.append(f"output{i}_f1")

sums = {col: 0.0 for col in columns}  # 每個欄位的累加和
counts = {col: 0 for col in columns}  # 每個欄位的計數器

# 讀取檔案並計算總和與計數
with open(input_file, 'r', newline="") as f_input:
    reader = csv.DictReader(f_input)  # 使用 DictReader 方便按欄位名稱讀取
    for row in reader:
        for col in columns:
            if row[col]:  # 確保欄位非空
                sums[col] += float(row[col])  # 累加欄位值
                counts[col] += 1  # 增加計數

# 計算平均值
averages = {col: (sums[col] / counts[col] if counts[col] > 0 else 0) for col in columns}

# 印出平均值
print("每個欄位的平均值：")
for col, avg in averages.items():
    print(f"{col}: {round(avg, 3)}")

每個欄位的平均值：
output1_precision: 0.64
output1_recall: nan
output1_f1: 0.693
output2_precision: 0.637
output2_recall: nan
output2_f1: 0.692
output3_precision: 0.647
output3_recall: nan
output3_f1: 0.696
output4_precision: 0.648
output4_recall: nan
output4_f1: 0.697
output5_precision: 0.636
output5_recall: nan
output5_f1: 0.692
output6_precision: 0.638
output6_recall: nan
output6_f1: 0.692
output7_precision: 0.663
output7_recall: nan
output7_f1: 0.71
output8_precision: 0.601
output8_recall: nan
output8_f1: 0.672
